In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,9329.02,-0.14,-0.03,-0.02,0.00,-0.00,0.0,-0.00
3,-0.14,1371.74,0.01,-0.01,0.00,-0.00,-0.0,0.00
4,-0.03,0.01,585.19,-0.00,0.01,0.00,-0.0,-0.00
5,-0.02,-0.01,-0.00,160.73,-0.00,-0.00,0.0,0.00
6,0.00,0.00,0.01,-0.00,38.41,0.00,0.0,0.00
7,-0.00,-0.00,0.00,-0.00,0.00,8.76,-0.0,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,1.9,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,0.34


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00002,0.00000,-0.00001,0.00001,-0.00003
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00005,0.00005,-0.00001,-0.00005
5,-0.00002,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
6,0.00000,0.00001,0.00005,-0.00001,1.00000,0.00002,0.00000,0.00006
7,-0.00001,-0.00001,0.00005,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00001,-0.00001,-0.00001,0.00002,0.00000,-0.00001,1.00000,0.00003
9,-0.00003,0.00000,-0.00005,0.00000,0.00006,-0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.21110979499333762

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.08847360e+07 2.31811011e+05 4.06630765e+04 2.95311682e+03
 1.58434916e+02 7.35791769e+00 2.63376006e-01 9.59928474e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999726,-0.021457,-0.008964,-0.002453,-0.000586,-0.000134,-0.000029,-0.000005
1,0.020555,0.995899,-0.085932,-0.018932,-0.004453,-0.001017,-0.000221,-0.000039
2,0.010569,0.084363,0.994415,-0.061014,-0.013311,-0.003018,-0.000657,-0.000118
3,0.003432,0.023579,0.058167,0.995871,-0.063991,-0.013606,-0.002940,-0.000525
4,0.001016,0.006914,0.016243,0.061641,0.994555,-0.080455,-0.016308,-0.002893
5,0.000306,0.002081,0.004871,0.017797,0.076993,0.990216,-0.113339,-0.018834
6,0.000098,0.000665,0.001556,0.005668,0.023753,0.106563,0.977814,-0.178676
7,0.000036,0.000243,0.000571,0.002075,0.008677,0.038069,0.175383,0.983723


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0002736088606978271,
 0.0041014540268048405,
 0.00558472369942975,
 0.0041285257707928125,
 0.00544462856906347,
 0.009784444894468614,
 0.022186061652042177,
 0.0162766837124243]